In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')

In [3]:
df_train = df_train.drop(['MiscFeature','Alley','Fence','PoolQC'],axis = 1)

In [4]:
df_test = df_test.drop(['MiscFeature','Alley','Fence','PoolQC'],axis = 1)

In [5]:
from sklearn.preprocessing import LabelEncoder
for i in df_train.columns:
    if df_train[i].dtype == object:
        lable = LabelEncoder()
        df_train[i] = lable.fit_transform(df_train[i])
        df_test[i] = lable.fit_transform(df_test[i])

In [6]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int64  
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   LotShape       1460 non-null   int64  
 7   LandContour    1460 non-null   int64  
 8   Utilities      1460 non-null   int64  
 9   LotConfig      1460 non-null   int64  
 10  LandSlope      1460 non-null   int64  
 11  Neighborhood   1460 non-null   int64  
 12  Condition1     1460 non-null   int64  
 13  Condition2     1460 non-null   int64  
 14  BldgType       1460 non-null   int64  
 15  HouseStyle     1460 non-null   int64  
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [7]:
df_train.corr()['SalePrice']

Id              -0.021917
MSSubClass      -0.084284
MSZoning        -0.166872
LotFrontage      0.351799
LotArea          0.263843
                   ...   
MoSold           0.046432
YrSold          -0.028923
SaleType        -0.054911
SaleCondition    0.213092
SalePrice        1.000000
Name: SalePrice, Length: 77, dtype: float64

In [8]:
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,65.0,8450,1,3,3,0,4,...,0,0,0,0,0,2,2008,8,4,208500
1,2,20,3,80.0,9600,1,3,3,0,2,...,0,0,0,0,0,5,2007,8,4,181500
2,3,60,3,68.0,11250,1,0,3,0,4,...,0,0,0,0,0,9,2008,8,4,223500
3,4,70,3,60.0,9550,1,0,3,0,0,...,272,0,0,0,0,2,2006,8,0,140000
4,5,60,3,84.0,14260,1,0,3,0,2,...,0,0,0,0,0,12,2008,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,3,62.0,7917,1,3,3,0,4,...,0,0,0,0,0,8,2007,8,4,175000
1456,1457,20,3,85.0,13175,1,3,3,0,4,...,0,0,0,0,0,2,2010,8,4,210000
1457,1458,70,3,66.0,9042,1,3,3,0,4,...,0,0,0,0,2500,5,2010,8,4,266500
1458,1459,20,3,68.0,9717,1,3,3,0,4,...,112,0,0,0,0,4,2010,8,4,142125


In [9]:
for i in df_train.columns:
    if len(list(df_train[i].value_counts().keys()))<50:
        df_train[i] = df_train[i].apply(lambda x : np.random.choice(a=list(df_train[i].value_counts().keys()), p=[df_train[i].value_counts()[j]/sum(df_train[i].value_counts()) for j in list(df_train[i].value_counts().keys())]) if type(x) == float else x)
    elif len(list(df_train[i].unique())) == int(len(df_train[i])):
        df_train = df_train.drop(i, axis=1)
    else:
        df_train[i] = df_train[i].interpolate()

In [13]:
for i in df_test.columns:
    if len(list(df_test[i].value_counts().keys()))<50:
        df_test[i] = df_test[i].apply(lambda x : np.random.choice(a=list(df_test[i].value_counts().keys()), p=[df_test[i].value_counts()[j]/sum(df_test[i].value_counts()) for j in list(df_test[i].value_counts().keys())]) if type(x) == float else x)
    elif len(list(df_test[i].unique())) == int(len(df_test[i])):
        df_test = df_test.drop(i, axis=1)
    else:
        df_test[i] = df_test[i].interpolate()

In [14]:
X = df_train.drop('SalePrice' , axis=1)
y = df_train['SalePrice']

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()

In [17]:
X = scaler.fit_transform(X)
df_test = scaler.transform(df_test)

In [18]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [35]:
hyper_param = {
    'n_estimators': [100],
    'criterion': ["squared_error"],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'min_weight_fraction_leaf': [0.0, 0.1],
    # 'max_leaf_nodes': [None, 10],
    # 'min_impurity_decrease': [0.0, 0.1],
    # 'bootstrap': [True, False],
    # 'oob_score': [False, True],
    # 'n_jobs': [None, -1],
    # 'random_state': [None, 42],
    # 'verbose': [0, 1],
    # 'warm_start': [False, True],
    # 'ccp_alpha': [0.0, 0.1],
    # 'max_samples': [None, 0.5]
}

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
full_model = GridSearchCV(model, hyper_param, cv=5, verbose=2)

In [38]:
full_model.fit(X, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100; total time=   2.2s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100; total time=   2.1s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100; total time=   2.1s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100; total time=   2.1s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100; total time=   2.1s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.1, n_estimators=100; total time

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'criterion': ['squared_error'],
                         'max_depth': [None, 10], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'min_weight_fraction_leaf': [0.0, 0.1],
                         'n_estimators': [100]},
             verbose=2)

In [39]:
full_model.best_params_

{'criterion': 'squared_error',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100}

In [40]:
y_pred = full_model.predict(df_test)

In [41]:
df_sub['SalePrice'] = y_pred

In [42]:
df_sub.to_csv('tneri_gin.csv', index=False, index_label=False)